<a href="https://colab.research.google.com/github/taka4ma/fastpages/blob/master/2022_03_06_How_to_easily_publish_a_Notebook_on_github_io_using_fastpages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# fastpagesを使ってgithub.ioでNotebookを簡単に公開する方法

> [Github Pages](https://pages.github.com/)でJupyterやGoogle ColaboratoryのNotebookを簡単に公開する方法を紹介します

- toc: true
- badges: false
- comments: false
- categories: [fastpages]

## 背景

Jupyter NotebookやGoogle ColaboratoryのNotebookファイルをGithub Pagesで公開する方法は、以下の2つが知られています。

1. NotebookファイルをMarkdownファイルへエクスポートしてGithubリポジトリへPushする。(GitHub PagesにはリポジトリにPushされたMarkdownファイルにアクセスすると、HTMLとしてレンダリングする機能があるため。)
1. NotebookファイルをGitHub GistにPushし、リポジトリにはgistの埋め込みスクリプトを記述したMarkdownかHTMLをPushする。(GitHub GistにはNotebookファイルを埋め込みスクリプトで表示するとHTMLとしてレンダリングする機能があるため。)

しかし、最初の方法はNotebookを作成・更新するたびに手動でMarkdownにエクスポートしなければなりません。Jupyter Notebookには開いているNotebookファイルをMarkdownファイルにコンバートしてダウンロードする機能があるので、それを利用することができます。Google ColaboratoryにはMarkdownファイルにコンバートする機能がないため、別に用意する必要があります。いずれにしても、NotebookファイルとMarkdownファイルの両方を管理しなければならないため、管理が煩雑になってしまいます。

また、2つ目の方法では公開するNotebookごとにgistを用意する必要があります。gistはそれ自体が1つのリポジトリなので、Notebookの数だけリポジトリを管理しなくてはならず、管理が煩雑になります。Google Colaboratoryには開いているNotebookファイルのコピーをGitHub Gistへ保存する機能がありますが、保存するたびに新しいgistとして保存されてしまうため、この機能を利用するならば、gistの埋め込みスクリプトを記述したファイルを都度更新しなくてはなりません。

このように、現在知られている2つの方法は、どちらもファイルやリポジトリの管理に手間がかかってしまうため、より簡単な方法が必要でした。


## fastpagesとは

[fastai/fastpages: An easy to use blogging platform, with enhanced support for Jupyter Notebooks\.](https://github.com/fastai/fastpages) にて公開されているソフトウェアです。使用するには公開されているリポジトリをCloneし、初期設定を行います。そしてそのリポジトリのmasterブランチへルールに従ってNotebookファイルをPushします。そうするとGithub Actionが実行されます。実行されたGithub Actionによりコンテナが起動し、リポジトリがコンテナ上にCloneされ、ファイルのコンバートが行われ、出来たファイルがリポジトリのgh-pagesブランチへPushされます。このリポジトリはgh-pagesブランチをGitHub Pagesのコンテンツとして公開するように設定されているので、コンバートされたファイルが公開されます。

## Initial settings
1. fastpagesのリポジトリ(https://github.com/fastai/fastpages) にブラウザでアクセスする.
1. Setup Instructionsに、`Generate a copy of this repo by clicking on this link.`と書かれているので、リンクをクリックする.
1. "Create a new repository from fastpages"というタイトルの、リポジトリ作成画面が表示されるので,
    1. `owner`に任意のアカウントまたはOrganizationsを選択する.
    3. `Repository name`にGithubの自分のユーザー名**以外**の名前を設定する.
    4. `Public/Private`は`Public`を選ぶ。
    5. `Include all branches`にチェックを**入れない**. (https://github.com/fastai/fastpages/issues/608)
    6. `Create repository from template`をクリックする.
1. ブラウザに作成したリポジトリが表示される
1. fastpagesのテンプレートからリポジトリを作成すると、GitHub Actionsのワークフローが実行され、PRが作成されるので、リポジトリのActionsタブを開いて、ワークフローが実行されているか確認する.
    - ワークフローが実行されていれば、All workflowsに実行されたワークフローが表示される.
    - もし、ワークフローが実行されていない場合は、リポジトリのSettings > Actions > General > Actions permissionsで実行が許可されているか確認する.
1. Pull requestsタブを開いて, "Inital Setup" というPRを開く.
1. PRの"Before you merge this PR"セクションを順番に処理する
    1. "Create an ssh key-pair. Open this utility. Select: RSA and 4096 and leave Passphrase blank. Click the blue button Generate-SSH-Keys." 
        1. ブラウザで[Online Generate SSH keys algorithm RSA,DSA,ECDSA](https://8gwifi.org/sshfunctions.jsp)を開く
        2. `SSH-Keygen Online Algorithm`で`RSA`を選択する.
        3. `RSA Key Size`で`4096`を選択する
        4. `Passphrase`は入力しない
        5. `Generate-SSH-Keys`をクリックする. そうすると、SSH鍵が作成され、表示される
        - ssh鍵は、ローカルマシンで、`ssh-keygen`コマンドで作成してもよい.
    1. "Navigate to this link and click New repository secret. Copy and paste the Private Key into the Value field. This includes the "---BEGIN RSA PRIVATE KEY---" and "--END RSA PRIVATE KEY---" portions. In the Name field, name the secret SSH_DEPLOY_KEY."
        1. リポジトリの, Settings > Secrets > Actions を開き、`New repository secret`をクリックする.
        1. `Name`に`SSH_DEPLOY_KEY`と入力.
        1. `Value`に生成した秘密鍵をペーストする. この時、"---BEGIN RSA PRIVATE KEY---" から "--END RSA PRIVATE KEY---"までを含むこと.
        1. `Add secret`をクリックして、登録する.
    1. "Navigate to this link and click the Add deploy key button. Paste your Public Key from step 1 into the Key box. In the Title, name the key anything you want, for example fastpages-key. Finally, make sure you click the checkbox next to Allow write access (pictured below), and click Add key to save the key."
        1. リポジトリの, Settings > Deploy keysを開き、`Add deploy key`をクリックする.
        1. `タイトル`に、fastpages-keyなど、好きな名前を付けます。
        1. 作成した公開鍵を`Key`にペーストする.
        1. `Allow write access`のチェックボックスにチェックをつける.
        2. `Add key`をクリックしてキーを保存する.
1. "Inital Setup"PRの`Merge pull request`をクリックし、コミットメッセージを入力するか、またはデフォルトのまま、`Confirm merge`をクリックする。
    - PRをマージすると、GitHub Actionsのワークフローが実行され、リポジトリの内容が変更される、最初のデプロイが行われる。
1. リポジトリのCodeタブを開く。
1. README.mdが変更され、fastpagesによってデプロイされるコンテンツのトップのURLが記述されているので、そのURLをリンクで開く
    - URLは通常、 https://ユーザ名.github.io/リポジトリ名
    - 404の場合は、デプロイがまだ終わっていないので、数分待ってからリロードする
    

## markdownファイルを投稿する

1. YYYY-MM-DD-*.mdの命名規則に従って、投稿するmarkdownファイルのファイル名を変更します。
1. `/_post` ディレクトリへプッシュしてください。


## Notebookファイルを投稿する

1. YYYY-MM-DD-*.mdの命名規則に従って、投稿するnotebookファイルのファイル名を変更します。
1. `/_notebooks` ディレクトリへプッシュしてください。
